# Benchmarks and Metrics with StudyRunner

Run a tiny multi-problem study and inspect hypervolume metrics. Keep budgets small so the notebook stays quick.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from vamos.study.runner import StudyRunner, StudyTask
from vamos.analysis.tuning_viz import study_results_to_dataframe
from vamos.core.experiment_config import ExperimentConfig

plt.style.use("ggplot")

In [ ]:
# Build a small set of StudyTasks (two problems, two algorithms, two seeds)
problems = [
    {"problem": "zdt1", "n_var": 12},
    {"problem": "zdt2", "n_var": 12},
]
algorithms = ["nsgaii", "moead"]
seeds = [0, 1]

tasks = []
for prob in problems:
    for alg in algorithms:
        for seed in seeds:
            tasks.append(
                StudyTask(
                    algorithm=alg,
                    engine="numpy",
                    problem=prob["problem"],
                    n_var=prob["n_var"],
                    seed=seed,
                    config_overrides={
                        "population_size": 30,
                        "offspring_population_size": 30,
                        "max_evaluations": 400,
                    },
                )
            )

runner = StudyRunner(verbose=True)
results = runner.run(tasks)
df = study_results_to_dataframe(results)
df.head()

In [ ]:
# Summarize HV per algorithm/problem
summary = df.groupby(["problem", "algorithm"]).agg({"hv": ["mean", "std"]})
summary.columns = ["hv_mean", "hv_std"]
summary = summary.reset_index()
summary

In [ ]:
# Simple bar plot of HV means
pivot = summary.pivot(index="problem", columns="algorithm", values="hv_mean")
pivot.plot(kind="bar", figsize=(6, 4))
plt.ylabel("Hypervolume (mean)")
plt.title("Tiny study: HV by algorithm")
plt.tight_layout()
plt.show()